# Importación de dependencias y lectura de datasets.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from tensorflow.keras.layers import Input, Dense, Concatenate, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Dataset Oracle-RedBull
races_info = pd.read_csv('final_data_spanish.csv')

# Definición de variables de entrada y salida

# EDA

In [ ]:
races_info = pd.DataFrame({
    'Stint': [1, 2, 1, 2, 3],  # Ejemplo de stints
    'lapNumberAtBeginingOfStint': [10, 30, 15, 40, 60],  # Vueltas de inicio del stint
    'Compound': ['Soft', 'Medium', 'Soft', 'Hard', 'Medium'],  # Tipo de neumático
    'designedLaps': [70, 70, 70, 70, 70]  # Vueltas totales de la carrera
})

# Crear `num_stops` calculando el número de stints únicos por carrera/piloto
races_info['num_stops'] = races_info_ds.groupby(['eventYear', 'Driver'])['Stint'].transform('max')

# Crear columnas para vueltas de parada
for i in range(1, races_info['num_stops'].max() + 1):
    races_info[f'lap_stop_{i}'] = races_info.apply(
        lambda row: row['lapNumberAtBeginingOfStint'] if row['Stint'] == i else np.nan, axis=1
    )

# Crear columnas para tipos de neumático en cada parada
for i in range(1, races_info['num_stops'].max() + 1):
    races_info[f'tire_type_{i}'] = races_info.apply(
        lambda row: row['Compound'] if row['Stint'] == i else np.nan, axis=1
    )

# Llenar los NaN con valores específicos para mantener consistencia de datos
races_info = races_info.fillna(method='ffill').fillna(method='bfill')

KeyError: 'eventYear'

In [ ]:
# Codificar variables categóricas de entrada

categorical_cols_input = ['EventName', 'Team', 'Driver', 'Rainfall']
encoder_input = OneHotEncoder(sparse_output=False)
encoded_input = encoder_input.fit_transform(races_info[categorical_cols_input])

# Variables numéricas de entrada
numeric_cols_input = ['RoundNumber', 'eventYear', 'meanAirTemp', 'meanTrackTemp',
                      'meanHumid', 'GridPosition', 'CircuitLength', 'designedLaps']
numeric_input = races_info[numeric_cols_input].values
print('Forma de numeric_input:', numeric_input.shape)
print('Forma de encoded_input:', encoded_input.shape)
# Combinar y escalar
X = np.hstack((numeric_input, encoded_input))
scaler_input = MinMaxScaler()
X = scaler_input.fit_transform(X)

Forma de numeric_input: (286, 8)
Forma de encoded_input: (286, 79)


In [ ]:
# Escalar vueltas de parada
scaler_lap_stops = MinMaxScaler(feature_range=(0, 1))
lap_stop_cols = [f'lap_stop_{i}' for i in range(1, races_info['num_stops'].max() + 1)]
races_info[lap_stop_cols] = scaler_lap_stops.fit_transform(races_info[lap_stop_cols])

# Codificar tipos de neumático
tire_type_cols = [f'tire_type_{i}' for i in range(1, races_info['num_stops'].max() + 1)]
encoder_tire_types = OneHotEncoder(sparse_output=False)
tire_types_encoded = encoder_tire_types.fit_transform(races_info[tire_type_cols])

# Combinar todas las columnas en un solo array de salida `y`
y = np.hstack([races_info[lap_stop_cols].values, tire_types_encoded])

# Arquitectura CGAN

In [ ]:
condition_dim = X.shape[1]
num_output_dim = 2 # Número de variables numéricas de salida
cat_output_dim = tire_types_encoded.shape[1]  # Número de categorías de 'Compound'
strategy_dim = num_output_dim + cat_output_dim
noise_dim = 100

In [ ]:
def build_generator(noise_dim, condition_dim, max_stops=3, num_tire_types=3):
    noise_input = Input(shape=(noise_dim,))
    condition_input = Input(shape=(condition_dim,))
    merged = Concatenate()([noise_input, condition_input])

    x = Dense(128)(merged)
    x = LeakyReLU(0.2)(x)
    x = Dense(256)(x)
    x = LeakyReLU(0.2)(x)

    # Salida para el número de paradas (1 a max_stops)
    num_stops_output = Dense(max_stops, activation='softmax')(x)  # Predice el número de paradas como una categoría

    # Salida para la vuelta de cada parada
    lap_stop_outputs = []
    for _ in range(max_stops):
        lap_stop = Dense(1, activation='sigmoid')(x)  # Escalar entre 0 y 1
        lap_stop_outputs.append(lap_stop)

    # Concatenar todas las vueltas de parada y escalar al rango de vueltas
    lap_stop_outputs = Concatenate()(lap_stop_outputs)
    lap_stop_outputs_scaled = lap_stop_outputs * (condition_input[:, -1:] - 1) + 1  # Escalar entre [1, designedLaps]

    # Salida para el tipo de neumático en cada parada
    tire_type_outputs = []
    for _ in range(max_stops):
        tire_type = Dense(num_tire_types, activation='softmax')(x)  # Predice el tipo de neumático en cada parada
        tire_type_outputs.append(tire_type)

    # Concatenar todas las salidas
    output = Concatenate()([num_stops_output, lap_stop_outputs_scaled] + tire_type_outputs)

    model = Model([noise_input, condition_input], output)
    return model

In [ ]:
# Discriminador
def build_discriminator(strategy_dim, condition_dim):
    strategy_input = Input(shape=(strategy_dim,))
    condition_input = Input(shape=(condition_dim,))
    merged = Concatenate()([strategy_input, condition_input])

    x = Dense(256)(merged)
    x = LeakyReLU(0.2)(x)
    x = Dense(128)(x)
    x = LeakyReLU(0.2)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = Model([strategy_input, condition_input], output)
    return model

In [ ]:
# Compilar modelos
# Crear el discriminador entrenable y compilarlo
discriminator = build_discriminator(strategy_dim, condition_dim)
optimizer_d = Adam(0.0002, 0.5)
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer_d, metrics=['accuracy'])

# Crear una copia del discriminador para usarla como no entrenable en el modelo combinado
discriminator_non_trainable = clone_model(discriminator)
discriminator_non_trainable.set_weights(discriminator.get_weights())
discriminator_non_trainable.trainable = False  # Configurar la copia como no entrenable

# Crear el generador normalmente
generator = build_generator(noise_dim, condition_dim, num_output_dim, cat_output_dim)

# Construir el modelo combinado usando el discriminador no entrenable
noise_input = Input(shape=(noise_dim,))
condition_input = Input(shape=(condition_dim,))
generated_strategy = generator([noise_input, condition_input])
validity = discriminator_non_trainable([generated_strategy, condition_input])

combined_model = Model([noise_input, condition_input], validity)
optimizer_g = Adam(0.0002, 0.5)
combined_model.compile(loss='binary_crossentropy', optimizer=optimizer_g)

# Entrenamiento del modelo

In [ ]:
epochs = 20
batch_size = 32
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

print("Pesos entrenables del discriminador:", len(discriminator.trainable_weights))
print("Pesos entrenables del generador:", len(generator.trainable_weights))
print("Pesos entrenables del modelo combinado:", len(combined_model.trainable_weights))

for epoch in range(epochs):
    # ---------------------
    #  Entrenar el discriminador
    # ---------------------
    idx = np.random.randint(0, X.shape[0], batch_size)
    real_strategies = y[idx]
    conditions = X[idx]

    noise = np.random.normal(0, 1, (batch_size, noise_dim))
    generated_strategies = generator.predict([noise, conditions])

    d_loss_real = discriminator.train_on_batch([real_strategies, conditions], valid)
    d_loss_fake = discriminator.train_on_batch([generated_strategies, conditions], fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ---------------------
    #  Entrenar el generador
    # ---------------------
    g_loss = combined_model.train_on_batch([noise, conditions], valid)

    print(f"{epoch} [Pérdida D: {d_loss[0]:.4f}, Precisión: {d_loss[1]*100:.2f}%] [Pérdida G: {g_loss:.4f}]")



Pesos entrenables del discriminador: 6
Pesos entrenables del generador: 8
Pesos entrenables del modelo combinado: 8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
0 [Pérdida D: 0.6892, Precisión: 57.81%] [Pérdida G: 0.6729]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1 [Pérdida D: 0.6987, Precisión: 47.79%] [Pérdida G: 0.6758]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2 [Pérdida D: 0.6978, Precisión: 48.85%] [Pérdida G: 0.6740]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
3 [Pérdida D: 0.6969, Precisión: 49.94%] [Pérdida G: 0.6724]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
4 [Pérdida D: 0.6961, Precisión: 50.26%] [Pérdida G: 0.6725]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
5 [Pérdida D: 0.6955, Precisión: 49.51%] [Pérdida G: 0.6730]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
6 [Pérdida D: 0.6946, Precisión: 49.57%] [Pérdida G: 0.6740]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
7 [Pérdida D: 0.6933, Precisión: 50.23%] [Pérdida G: 0.6731]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
8 [Pérdida D: 0.6923, Precisión: 50.64%] [Pérdida G: 0

In [ ]:
# Generar una estrategia
noise = np.random.normal(0, 1, (1, noise_dim))
sample_condition = X[0:1]  # Condiciones para un circuito específico
generated_strategy = generator.predict([noise, sample_condition])

max_stops = 3

# Interpretar la salida del generador
num_stops = np.argmax(generated_strategy[:, :max_stops]) + 1  # Número de paradas

lap_stops = generated_strategy[:, max_stops:max_stops + num_stops]  # Vueltas de parada escaladas
lap_stops = lap_stops.astype(int)  # Redondear a vueltas enteras

num_tire_types = races_info['Compound'].nunique()

tire_types = []
start_idx = max_stops + num_stops
for i in range(num_stops):
    tire_type_index = np.argmax(generated_strategy[:, start_idx + i*num_tire_types:start_idx + (i+1)*num_tire_types])
    tire_type = encoder_tire_types.categories_[0][tire_type_index]  # Mapeo del tipo de neumático
    tire_types.append(tire_type)

# Imprimir la estrategia generada
print("Número de paradas:", num_stops)
for i in range(num_stops):
    print(f"Parada {i+1}:")
    print(f" - Vuelta de parada: {lap_stops[0][i]}")
    print(f" - Tipo de neumático: {tire_types[i]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
Estrategia generada:
Vuelta de parada: 35.68
Duración del stint: 29.31
Compuesto de neumático: HARD
